### Importierte Biblotheken

In [4]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path
import json
import lzma
import pandas as pd



#### Voraussetzungen
* **Bibliotheken:** `kaggle`, `os`, `shutil` (`pip install kaggle`).
* **Kaggle API Key:** Eine `kaggle.json` API-Schlüsseldatei im Verzeichnis `~/.kaggle/` ist **zwingend erforderlich**. 
Ohne sie schlägt die Authentifizierung im Download-Schritt fehl. Details zur Einrichtung finden Sie in der [Kaggle Dokumentation](https://www.kaggle.com/docs/api) oder im Projektverzeichnis in der README.md Datei.


### Prüfung, Download 

Die nächste Code-Zelle führt die Kernlogik aus:

1.  **Prüfung:** Testet, ob der Datensatz bereits am `data_dir` existiert und Inhalt hat.
2.  **Download (falls nötig):**
    * **Kaggle API:** Die `kaggle` API wird initialisiert und authentifiziert sich über Ihre `~/.kaggle/kaggle.json`-Datei.
    * **Download & Entpacken:** `api.dataset_download_files(...)` lädt den Datensatz mit der `DATASET_ID` herunter und entpackt (`unzip=True`) dessen Inhalt direkt in den `data_dir`.


In [5]:
DATASET_ID = "thecoderenroute/instagram-posts-dataset"

base_dir = Path.cwd()
if base_dir.name == "notebooks":
    base_dir = base_dir.parent
    print(f"Aktueller Pfad: {base_dir}")

data_dir = base_dir / "daten" / "rohdaten" / "instagram-posts-dataset"
os.makedirs(data_dir, exist_ok=True)

data_exists = False
valid_files = [f for f in os.listdir(data_dir) if not f.startswith('.')]
if os.path.exists(data_dir) and valid_files:
    try:
        if len(valid_files) > 0:
             data_exists = True
             print(f"Datensatz bereits vorhanden unter: {data_dir}")
        else:
             print(f"Hinweis: Zielordner '{data_dir}' existiert, ist aber leer.")
    except OSError:
         print(f"Hinweis: Pfad '{data_dir}' existiert, aber Inhalt konnte nicht gelesen werden.")

# ___DOWNLOAD___
if not data_exists:
    print(f"Datensatz nicht oder nur leer vorhanden unter '{data_dir}'. Starte Download via Kaggle API...")
    try:
        print("Initialisiere Kaggle API...")
        api = KaggleApi()
        api.authenticate() 
        print("Authentifizierung erfolgreich.")
        print(f"Lade '{DATASET_ID}' nach '{data_dir}' herunter und entpacke...")
        api.dataset_download_files(DATASET_ID, path=data_dir, unzip=True)
        print("Download und Entpacken abgeschlossen.")

    except ImportError:
         print(f"**FEHLER:** Bibliothek 'kaggle' nicht gefunden. Bitte installieren: pip install kaggle")
    except Exception as e:
        print(f"**FEHLER** beim Download via Kaggle API: {e}")
else:
    print("Datensatz bereits vorhanden.")
for file in os.listdir(data_dir):
    print(f"Gefundene Datei: {file}")

Datensatz bereits vorhanden unter: /Users/zhanna97yan/Library/CloudStorage/OneDrive-SEPadersoftGmbH&Co.KG/Dokumente/FHDW/FHDW Projects/ADA_Instagram_Like_Vorhersage/daten/rohdaten/instagram-posts-dataset
Datensatz bereits vorhanden.
Gefundene Datei: .DS_Store
Gefundene Datei: Data


### Dataframe erstellen
- die Daten aus dem "node"-Schlüssel jeder JSON-Datei zu extrahieren und als Dataset speichern
   - Alle benötigten Werten in den JSON-Dateien sind in einem "node"-Schlüssel gespeichert
- die extrahierten Daten in ein Pandas DataFrame umwandeln



In [6]:
all_data = []
# durch alle Unterordner iterieren
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".json.xz"):
            file_path = os.path.join(root, file)
            try:
                # Entpacken und Lesen der JSON.xz-Datei
                with lzma.open(file_path, mode='rt', encoding='utf-8') as f:
                    data = json.load(f)
                    if "node" in data:
                        all_data.append(data["node"])
            except Exception as e:
                print(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")

# DataFrame aus den gesammelten Daten
if all_data:
    df = pd.DataFrame(all_data)
    print("DataFrame erfolgreich erstellt.")
    df.head()
else:
    print("Keine Daten gefunden.")

DataFrame erfolgreich erstellt.


### Fehlende Werte entfernen
- die Spalten mit mehr als 99% fehlenden Werten entfernen

In [7]:
missing_ratio = df.isnull().mean()
cols_99_missing = missing_ratio[missing_ratio >= 0.99].index.tolist()

print("Spalten mit mindestens 99% fehlenden Werten:")
for col in cols_99_missing:
     print(f"- {col}")

df = pd.DataFrame(df.drop(columns=cols_99_missing))
df



Spalten mit mindestens 99% fehlenden Werten:
- gating_info
- fact_check_overall_rating
- fact_check_information
- media_overlay_info
- sensitivity_friction_info
- accessibility_caption
- nft_asset_info


,__typename,id,sharing_friction_info,dimensions,display_url,display_resources,is_video,media_preview,tracking_token,edge_media_to_tagged_user,...,viewer_has_saved_to_collection,viewer_in_photo_of_you,viewer_can_reshare,thumbnail_src,thumbnail_resources,edge_sidecar_to_children,has_upcoming_event,edge_liked_by,coauthor_producers,pinned_for_users
0,GraphSidecar,2868439159916464863,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1350, 'width': 1080}",https://instagram.fbom3-2.fna.fbcdn.net/v/t51....,[{'src': 'https://instagram.fbom3-2.fna.fbcdn....,False,None,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': [{'node': {'user': {'full_name': 's ...,...,False,False,True,https://instagram.fbom3-2.fna.fbcdn.net/v/t51....,[{'src': 'https://instagram.fbom3-2.fna.fbcdn....,{'edges': [{'node': {'__typename': 'GraphImage...,NaN,NaN,NaN,NaN
1,GraphSidecar,3066886135067171352,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1350, 'width': 1080}",https://instagram.fbom3-1.fna.fbcdn.net/v/t51....,NaN,False,None,NaN,{'edges': [{'node': {'user': {'full_name': 'su...,...,NaN,NaN,True,https://instagram.fbom3-1.fna.fbcdn.net/v/t51....,[{'src': 'https://instagram.fbom3-1.fna.fbcdn....,{'edges': [{'node': {'__typename': 'GraphImage...,False,{'count': 7650},[],[]
2,GraphSidecar,3029832820659598377,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1350, 'width': 1080}",https://instagram.fbom19-1.fna.fbcdn.net/v/t51...,[{'src': 'https://instagram.fbom19-1.fna.fbcdn...,False,None,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': [{'node': {'user': {'full_name': 'Di...,...,False,False,True,https://instagram.fbom19-1.fna.fbcdn.net/v/t51...,[{'src': 'https://instagram.fbom19-1.fna.fbcdn...,{'edges': [{'node': {'__typename': 'GraphImage...,NaN,NaN,NaN,NaN
3,GraphSidecar,2889470489057495466,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1350, 'width': 1080}",https://scontent-bom1-1.cdninstagram.com/v/t51...,[{'src': 'https://scontent-bom1-1.cdninstagram...,False,None,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': []},...,False,False,True,https://scontent-bom1-1.cdninstagram.com/v/t51...,[{'src': 'https://scontent-bom1-1.cdninstagram...,{'edges': [{'node': {'__typename': 'GraphImage...,NaN,NaN,NaN,NaN
4,GraphImage,3053766391959056433,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1080, 'width': 1080}",https://instagram.fbom19-2.fna.fbcdn.net/v/t51...,[{'src': 'https://instagram.fbom19-2.fna.fbcdn...,False,ACoq0JrkISACzDPAB7epqKG6LRGWTgKcZGfbsfrVTUGIkA...,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': [{'node': {'user': {'full_name': 'KL...,...,False,False,True,https://instagram.fbom19-2.fna.fbcdn.net/v/t51...,[{'src': 'https://instagram.fbom19-2.fna.fbcdn...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,GraphSidecar,3006774447044259315,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1350, 'width': 1080}",https://instagram.fbom3-2.fna.fbcdn.net/v/t51....,[{'src': 'https://instagram.fbom3-2.fna.fbcdn....,False,None,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': [{'node': {'user': {'full_name': 'An...,...,False,False,True,https://instagram.fbom3-2.fna.fbcdn.net/v/t51....,[{'src': 'https://instagram.fbom3-2.fna.fbcdn....,{'edges': [{'node': {'__typename': 'GraphImage...,NaN,NaN,NaN,NaN
1964,GraphImage,3066434777540625472,"{'should_have_sharing_friction': False, 'bloks...","{'height': 1350, 'width': 1080}",https://instagram.fbom19-1.fna.fbcdn.net/v/t51...,[{'src': 'https://instagram.fbom19-1.fna.fbcdn...,False,ACIq1pJRH169hVJriRjj7o9v8T1pJkZiHycnPHrxxjrzSI...,eyJ2ZXJzaW9uIjo1LCJwYXlsb2FkIjp7ImlzX2FuYWx5dG...,{'edges': []},...,False,False,True,https://instagram.fbom19-1.fna.fbcdn.net/v/t51...,[{'src': 'https://instagram.fbom19-1.fna.fbcdn...,NaN,NaN,NaN,NaN,NaN
1965,GraphSidecar,2791858705941864416,"{'should_have_sharing_friction': False, 'bloks...

### Ladezeit und Arbeitsspeicher sparen 
- DataFrame temporär als Pickle-Datei speichern und im neuen Notebook laden, ohne die JSON.xz-Dateien erneut zu verarbeiten
- In der Regel ist die Pickle-Datei kleiner als die JSON.xz-Datei

#### In neuen Notebook importieren
```python
import pandas as pd
df = pd.read_pickle("instagram_node_dataframe.pkl")
```

In [8]:
df.to_pickle("instagram_node_dataframe.pkl")
